In [2]:
pip install pydeck

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import pydeck as pdk
#버스정류장 위치정보 data
df_coord = pd.read_csv('bus_coord.csv',sep=",", encoding='cp949')
df_coord
lon = df_coord['X좌표']
lat = df_coord['Y좌표']

templist=[]
for i in range(len(lon)):    
    tempcoord=[lon[i],lat[i]]
    templist.append(tempcoord)

df_coord['coordinates'] = templist
df_coord

#df_coord2 = df_coord[['정류소번호','coordinates']]
#df_coord2

#정류소번호 및 노선번호
df_foo = pd.read_csv('bus_line.csv',sep=',', encoding='cp949')
df_foo.head()
df_foo_1 = df_foo[['노선번호','버스정류장ARS번호']]
df_foo_1.columns=['노선번호','정류소번호']
df_foo_1

#합병
df_merge=pd.merge(left=df_foo_1, right = df_coord, how ="outer", on = '정류소번호')
df_merge

#행에서 nans제거
df_merge=df_merge.dropna(subset=['X좌표'])
df_merge=df_merge.dropna(subset=['Y좌표'])
df_merge=df_merge.dropna(subset=['노선번호'])
df_merge=df_merge.dropna(subset=['정류소번호'])
df_merge=df_merge.dropna(subset=['coordinates'])
df_merge

#특정노선번호
contains_753 = df_merge['노선번호'].str.contains('753')
line_753 = df_merge[contains_753]
df_1 = line_753.drop_duplicates(['정류소번호'],keep='first')
df_1

point_layer = pdk.Layer(
    'ScatterplotLayer',
    df_merge,
    get_position = '[X좌표,Y좌표]',
    get_radius =50,
    get_fill_color = '[255,255,255]',
    pickable = True,
    auto_highlight=True
)

center = [126.986,37.565]
view_state = pdk.ViewState(longitude=center[0], latitude=center[1], zoom=10)

r = pdk.Deck(layers=[point_layer],initial_view_state=view_state)
r.to_html("test_for_bus_stop.html")

FileNotFoundError: [Errno 2] No such file or directory: 'bus_coord.csv'

In [6]:
import pandas as pd
import pydeck as pdk
#버스정류장 위치정보 data
df_coord = pd.read_csv('bus_coord.csv',sep=",", encoding='cp949')
df_coord
lon = df_coord['X좌표']
lat = df_coord['Y좌표']

templist=[]
for i in range(len(lon)):    
    tempcoord=[lon[i],lat[i]]
    templist.append(tempcoord)

df_coord['coordinates'] = templist
df_coord

,정류소번호,정류소명,X좌표,Y좌표,coordinates
0,?01001,종로2가사거리,126.987750,37.569765,"[126.9877498816, 37.569765125100005]"
1,01002,창경궁.서울대학교병원,126.996566,37.579183,"[126.9965660023, 37.5791830159]"
2,01003,명륜3가.성대입구,126.998340,37.582671,"[126.99834010040001, 37.5826711749]"
3,01004,종로2가.삼일교,126.987613,37.568579,"[126.9876130976, 37.5685792736]"
4,01005,혜화동로터리.여운형활동터,127.001744,37.586243,"[127.00174399999999, 37.586242999999996]"
...,...,...,...,...,...
11275,25995,우성아파트,127.139339,37.550386,"[127.1393385419, 37.550385731999995]"
11276,25996,우성아파트,127.140046,37.550643,"[127.1400463819, 37.550642607600004]"
11277,25997,조일약국,127.123596,37.533630,"[127.12359618200001, 37.533630263]"
11278,25998,성내시장,127.125497,37.536155,"[127.1254968434, 37.536154934]"


In [7]:

#정류소번호 및 노선번호
df_foo = pd.read_csv('bus_line.csv',sep=',', encoding='cp949')
df_foo.head()
df_foo_1 = df_foo[['노선번호','버스정류장ARS번호']]
df_foo_1.columns=['노선번호','정류소번호']
df_foo_1


,노선번호,정류소번호
0,N13,11284
1,100,11428
2,100,11387
3,100,11374
4,100,11373
...,...,...
465032,9714,63675
465033,9714,63660
465034,9714,63658
465035,9714,63656


In [8]:
# 노선 이름 데이터 정리
df_foo_2 = df_foo_1[['노선번호']]
df_foo_3 = df_foo_2.drop_duplicates('노선번호')
df_foo_3

line_num=[]
for i in range(len(df_foo_3)):
    line = df_foo_3.iloc[i,0]
    line_num.append(line)

df_geometry = pd.DataFrame(columns=['line_name'])
df_geometry['line_name'] = line_num
df_geometry

df_line = df_geometry.sort_values(by = ['line_name'])
df_line

,line_name
473,0017
223,02
62,03
178,05
1,100
...,...
398,종로11
418,종로12
413,종로13
421,중랑01


In [9]:
#데이터 묶기
eachgroup = list(df_merge.groupby('노선번호'))
coordinate_dummy=[]
for i in range(len(eachgroup)):
    df_line_nums = eachgroup[i][1]
    df_line_num = df_line_nums.drop_duplicates(['정류소번호'],keep='first')
    series_coords = df_line_num['coordinates']
    arr_coords = series_coords.array
    list_coords = list(arr_coords)
    coordinate_dummy.append(list_coords)
type(coordinate_dummy)


list

In [10]:
eachgroup = list(df_merge.groupby('노선번호'))
#a=[]
test=eachgroup[0][1]
test
test_1 = test.drop_duplicates(['정류소번호'],keep='first')
test_1
test_2 = test_1['coordinates']
test_2
test_array=test_2.array
test_array
test_list = list(test_array)
test_list
type(test_list)

list

In [11]:
import re
def make_color(x): 
    X =re.findall("\d+",x)
    Y= float(X[0])
    z=""
    if 0<Y<10 : z = [255,255,0]
    elif 10<Y<1000: z = [0,0,255]
    elif 1000<Y<9000: z = [0,153,0] 
    elif 9000<Y<10000: z = [255,0,0]  
    return z

df_line['coordinates'] = coordinate_dummy
df_line
df_line['color'] = df_line['line_name'].apply(make_color)
df_line

,line_name,coordinates,color
473,0017,"[[126.9631917349, 37.5240898013], [126.9662070...","[0, 0, 255]"
223,02,"[[126.9924839556, 37.560871736399996], [126.99...","[255, 255, 0]"
62,03,"[[126.9784645011, 37.558492025700005], [127.00...","[255, 255, 0]"
178,05,"[[126.98524075030001, 37.5606538896], [126.992...","[255, 255, 0]"
1,100,"[[127.07005600000001, 37.63813], [127.06893472...","[0, 0, 255]"
...,...,...,...
398,종로11,"[[126.9765652658, 37.5726577416], [126.9762857...","[0, 0, 255]"
418,종로12,"[[126.9973383362, 37.5745325055], [127.0019395...","[0, 0, 255]"
413,종로13,"[[126.96170682350001, 37.598133066500004], [12...","[0, 0, 255]"
421,중랑01,"[[127.06771591629999, 37.602571244699995], [12...","[255, 255, 0]"


In [1]:
line_layer = pdk.Layer(
    'PathLayer',
    df_line,
    get_path='coordinates',
    get_width = 10,
    get_color='[color[0],color[1],color[2]]',
)

center = [126.986,37.565]
view_state = pdk.ViewState(longitude=center[0], latitude=center[1], zoom=10)

r = pdk.Deck(layers=[line_layer],initial_view_state=view_state)
r.to_html("test_for_bus_stop.html")


NameError: name 'pdk' is not defined

In [13]:
import re

def make_color(x): 
    num =re.findall("\d+",x)
    Y= float(num[0])
    if 10<Y<1000: color = '[0,0,255]'
    elif 1000<Y<9000: color = '[0,153,0]' 
    elif 9000<Y<10000: color = '[255,0,0]' 
    elif 0<Y<10 : color = '[255,255,0]'
    return color
make_color('종로05')

'[255,255,0]'